In [15]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from matplotlib import cm as CM
import numpy as NP
import tensorflow as tf

In [16]:
tokens = pd.read_csv (r'Traits.csv')
sales = pd.read_csv (r'Sales.csv')

In [17]:
tokens

,Unnamed: 0,collection,contract,token_id,nft_url,trait_type,value
0,0,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Type,Human
1,1,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Hair,Water
2,2,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Clothing,Pink Oversized Kimono
3,3,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Eyes,Striking
4,4,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Mouth,Frown
...,...,...,...,...,...,...,...
696003,44435,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Background,Blue
696004,44436,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Skin,Olive Green
696005,44437,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Body,Puffer Green
696006,44438,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Face,Handlebar Bear


In [18]:
print(pd.unique(tokens['token_id']))
print(tokens.groupby('collection')['token_id'].nunique())

[    0     1     2 ... 19997 19999 20000]
collection
BAYC              10000
MAYC              18502
azuki             10000
clone_x           18895
cool_cats          9933
doodles           10000
meebits           20000
pudgy_penguins     8888
Name: token_id, dtype: int64


In [19]:
grouped_tokens = tokens.groupby('collection')
bayc = grouped_tokens.get_group('BAYC')
mayc = grouped_tokens.get_group('MAYC')
azuki = grouped_tokens.get_group('azuki')
clone_x = grouped_tokens.get_group('clone_x')
cool_cats = grouped_tokens.get_group('cool_cats')
doodles = grouped_tokens.get_group('doodles')
meebits = grouped_tokens.get_group('meebits')
pudgy_penguins = grouped_tokens.get_group('pudgy_penguins')

In [20]:
grouped_sales = sales.groupby('tokenSymbol')
bayc_sales = grouped_sales.get_group('BAYC')
mayc_sales = grouped_sales.get_group('MAYC')
azuki_sales = grouped_sales.get_group('AZUKI')
clone_x_sales = grouped_sales.get_group('CloneX')
cool_cats_sales = grouped_sales.get_group('COOL')
doodles_sales = grouped_sales.get_group('DOODLE')
meebits_sales = grouped_sales.get_group('⚇')
pudgy_penguins_sales = grouped_sales.get_group('PPG')

In [21]:
# bayc
print(bayc.groupby('trait_type')['token_id'].nunique())
print(bayc.groupby('trait_type')['value'].nunique())
print(pd.unique(bayc.groupby('trait_type').get_group('Background')['value']))
print(bayc.groupby('trait_type').get_group('Background')['value'].value_counts()/10000)

trait_type
Background    10000
Clothes        8114
Earring        2977
Eyes          10000
Fur           10000
Hat            7744
Mouth         10000
Name: token_id, dtype: int64
trait_type
Background     8
Clothes       43
Earring        6
Eyes          23
Fur           19
Hat           36
Mouth         33
Name: value, dtype: int64
['Orange' 'Aquamarine' 'Purple' 'Blue' 'Army Green' 'Yellow' 'Gray'
 'New Punk Blue']
Purple           0.1291
Yellow           0.1283
Orange           0.1273
Aquamarine       0.1266
Army Green       0.1243
Blue             0.1242
New Punk Blue    0.1232
Gray             0.1170
Name: value, dtype: float64


In [22]:
sales[sales['amount'] != 0]

,blockNumber,timeStamp,hash,from,contractAddress,to,tokenID,tokenName,tokenSymbol,gasPrice,currency,amount,amountUsd
200,13992202,1642010411,0xe3034e18aa6fa00ac39ffb6904baf9570cc31c1716c5...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xdec08cb92a506b88411da9ba290f3694be223c26,200,Azuki,AZUKI,619374837712,ETH,1.000000,3384.137681
201,13992202,1642010411,0xe3034e18aa6fa00ac39ffb6904baf9570cc31c1716c5...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xdec08cb92a506b88411da9ba290f3694be223c26,201,Azuki,AZUKI,619374837712,ETH,1.000000,3384.137681
202,13992202,1642010411,0xe3034e18aa6fa00ac39ffb6904baf9570cc31c1716c5...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xdec08cb92a506b88411da9ba290f3694be223c26,202,Azuki,AZUKI,619374837712,ETH,1.000000,3384.137681
203,13992202,1642010411,0xe3034e18aa6fa00ac39ffb6904baf9570cc31c1716c5...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xdec08cb92a506b88411da9ba290f3694be223c26,203,Azuki,AZUKI,619374837712,ETH,1.000000,3384.137681
204,13992202,1642010411,0xe3034e18aa6fa00ac39ffb6904baf9570cc31c1716c5...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xdec08cb92a506b88411da9ba290f3694be223c26,204,Azuki,AZUKI,619374837712,ETH,1.000000,3384.137681
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431366,14471743,1648430919,0xb362fed9aece8fa0ce819778062ddbb05533a68a6401...,0x88341d1a8f672d2780c8dc725902aae72f143b0c,0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,0xf6a325cef74fb60a685bdd5a79ded1ffe45996e8,550,CloneX,CloneX,23623743142,ETH,5.745767,18978.246953
431368,14471918,1648433160,0xcf8b9cf7a66f81ca0ca7648e4afa9183f171fb286bd8...,0xf6a325cef74fb60a685bdd5a79ded1ffe45996e8,0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,0x88341d1a8f672d2780c8dc725902aae72f143b0c,550,CloneX,CloneX,25970855725,ETH,11.000000,36254.122921
431378,14472353,1648438868,0xb7e933b307f2aee8795311d6226e3f656aeee5d0683a...,0x23324ed44904260fe555b18e5ba95c6030b9227d,0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,0xac7ebc289354503e1ddda06f30e75ca688382051,11018,CloneX,CloneX,38783642325,ETH,14.990000,49689.068827
431396,14473259,1648451159,0x695045f3093a9ee40769135159118f1fc2bd603a22a7...,0x24415add68ea0d8c6efa9ff6e4ca213417969d38,0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,0x2310abf1ef6beb30b74bff98ac1b326b647f4224,911,CloneX,CloneX,23543438533,ETH,16.000000,53186.027771


In [23]:
print(sales.groupby('tokenSymbol')['tokenID'].nunique())

tokenSymbol
AZUKI     10000
BAYC      10000
COOL       9938
CloneX    18554
DOODLE     9784
MAYC      17990
PPG        8887
⚇         19999
Name: tokenID, dtype: int64


In [24]:
bayc_features = bayc.groupby('token_id')#.filter('trait_type')
print(bayc_features)

In [25]:
def get_features(df):
    """
    :param df: dataframe to have features extracted
    :return: dataframe with rows of token id and columns of features
    """
    columns = {}

    for trait in df['trait_type'].unique():
        columns[trait] = ['None' for i in range(df.token_id.nunique())]

    for id in df['token_id'].unique():
        for _, row in df[df.token_id == id].iterrows():
                columns[row.trait_type][id] = row.value
    #%%

    return pd.DataFrame(columns)

In [26]:
def process_sales(df):
    df = df[df.amount != 0]
    df = df.sort_values(['tokenID', 'timeStamp'], ascending=[True, False]).drop_duplicates('tokenID')
    # print(df['tokenID'], df['timeStamp'], df['amount'])
    df = df[df.amount < 1e4]
    return df

# Kmeans model

In [27]:
bayc_sales_processed = process_sales(bayc_sales)
token_ids = bayc_sales_processed['tokenID'].unique()
prices = bayc_sales_processed.groupby('tokenID')['amount'].max()
bayc_features = get_features(bayc)
one_hot_bayc = pd.get_dummies(bayc_features.iloc[token_ids])
train, test = train_test_split(one_hot_bayc, test_size=0.2)
model = MiniBatchKMeans(n_clusters=100, batch_size=2048).fit(train)

In [38]:
test_indices = test.index.to_list()
test_prices = bayc_sales_processed.loc[bayc_sales_processed['tokenID'].isin(test_indices)]['amount']

In [32]:
print(model.predict(train))


[41  4 59 ... 14 56  6]


In [33]:
train_indices = train.index.to_list()
train_prices = bayc_sales_processed.loc[bayc_sales_processed['tokenID'].isin(train_indices)]['amount']
tokens_in_cluster = [0 for i in range(0, 100)]
price_per_cluster = [0 for i in range(0, 100)]
for index, item in enumerate(model.predict(train)):
    tokens_in_cluster[item] += 1
    price_per_cluster[item] += train_prices.iloc[index]

In [36]:
price_per_cluster = [price_per_cluster[i]/tokens_in_cluster[i] for i in range(0, 100)]
print(price_per_cluster)

[0.31614820721830605, 0.7527575308641976, 0.24610974816709283, 0.13010526939627395, 0.3052806414098655, 0.3520267094017095, 0.2139286507257425, 0.4507546252230814, 0.3115282462549856, 0.31501819485452615, 0.42281946313476565, 0.1833044176585553, 0.18607435570834976, 0.355396983092893, 0.21611184921858273, 0.19473409447989903, 0.3604476638965835, 0.4938896617465177, 0.24145741299009368, 0.45431086464463577, 0.30843189021144835, 0.29956332721606654, 0.46126895119907657, 0.7039861753488609, 0.5296530408163265, 0.35956137262539617, 0.3059700424325206, 0.22006881648479962, 0.27679005190311423, 0.40985085381872766, 0.29242328587906746, 0.4014832687974224, 0.6734807535709604, 0.41408881776239914, 1.1832252765043265, 0.44510139757038975, 0.4477908179012345, 0.664244072792516, 0.36747845627066505, 0.3463987433709956, 0.345376791111111, 0.6468866390306122, 0.4205375242384439, 0.23902834946214538, 0.3971405925339924, 0.2856038719700386, 0.28436059968639865, 0.5532846723377441, 0.35569343902850403

In [40]:
test_prices_predicted = [price_per_cluster[i] for i in model.predict(test)]
error = []

for index, item in enumerate(test_prices_predicted):
    error.append((item - test_prices.iloc[index])**2)

print(error)
print("MeanSquaredError: ", sum(error)/len(error))

[57.26222127751202, 0.06891756092203312, 22412.358525614403, 0.05594566009928677, 1568.2950573606984, 3751.2674423208846, 2999.487505814988, 46.838873145940525, 4153.379116524738, 0.026068496209453677, 0.17666845108989762, 48319.37965673977, 2654.6411633556695, 0.05106246007695514, 16822.203273110834, 0.15594184825858937, 13154.720507907386, 10342.39710952451, 2556.30295516973, 6335.923865807537, 1194.7236516860055, 6562.719954118973, 0.0025105380212731873, 1.4124651094497502, 1390.992647208569, 2576.6762429067076, 0.2954753349820031, 703.6827211985204, 0.045123652383660014, 1059.646322635115, 46.43404728560237, 0.14010862339101482, 587.9287673570628, 2995.221743789991, 7364.505240672261, 4243.780677309835, 1244.4012098390883, 9268.431269072717, 0.2989412866836008, 8663.238435030877, 11.14206568874758, 4743.307890774586, 2654.125485298196, 0.15597107498192822, 2458.762587772756, 22238.86770684694, 252.9120866532477, 0.05218112846557518, 0.271776151956337, 2136.504888247695, 19.90026806